In [14]:
import panel as pn
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from dotenv import load_dotenv
from panel.interact import interact
import requests
import numpy as np
import alpaca_trade_api as tradeapi
pn.extension('plotly')
%matplotlib inline

In [2]:
cities = ['PHXRNSA','ATXRSA','BOXRSA','NYXRSA','DAXRSA','SEXRNSA','CHXRSA','MIXRNSA','POXRSA','CEXRSA','WDXRSA','DNXRSA', 'CSUSHPISA']
dfs = []
for city in cities:
    city_url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id={city}&scale=left&cosd=2010-01-01&coed=2020-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2020-08-15&revision_date=2020-08-15&nd=1987-01-01"
    df = pd.read_csv(city_url, index_col='DATE')
    dfs.append(df)
df = pd.concat(dfs, axis=1)
df.rename(columns={'PHXRNSA': 'Phoenix', 'ATXRSA': 'Atlanta', 'BOXRSA': 'Boston', 'NYXRSA': 'New York', 'DAXRSA': 'Dallas', 'SEXRNSA': 'Seattle', 'CHXRSA': 'Chicago', 'MIXRNSA': 'Miami', 'POXRSA': 'Portland', 'CEXRSA': 'Cleveland', 'WDXRSA': 'Washington D.C.', 'DNXRSA': 'Denver', 'CSUSHPISA': 'U.S.A'}, inplace=True)
df

,Phoenix,Atlanta,Boston,New York,Dallas,Seattle,Chicago,Miami,Portland,Cleveland,Washington D.C.,Denver,U.S.A
DATE,,,,,,,,,,,,,
2010-01-01,111.764850,109.270010,154.978285,172.358995,119.617775,145.091980,126.839270,148.320830,149.580218,105.231744,179.971300,127.894429,147.395
2010-02-01,110.112470,108.722579,154.733077,172.444301,118.008395,143.563300,126.222254,147.516280,146.848143,104.609008,180.451787,127.728990,145.631
2010-03-01,109.519890,107.021328,155.617488,172.273041,118.314823,143.715810,125.022296,146.146380,146.906833,106.707658,179.883445,128.344074,145.858
2010-04-01,110.060030,107.612785,156.040202,171.958539,119.211158,145.138790,124.272016,145.040740,148.347588,106.428260,181.982927,129.031422,146.400
2010-05-01,111.000260,107.826431,156.410667,172.287204,119.407419,146.818040,124.041924,146.329820,147.862547,105.766944,182.984306,128.300995,146.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01,200.048255,156.930763,226.813721,203.627646,195.075463,256.144476,144.554834,247.364745,244.592416,127.782040,237.858167,226.473431,214.680
2020-02-01,201.435466,157.464403,227.729786,204.793983,194.873127,259.646542,145.111902,248.678833,246.196458,128.890384,238.828713,227.647558,215.716
2020-03-01,203.575201,157.811258,228.948595,205.297634,195.141653,266.194961,146.011528,250.442823,246.855602,130.075735,240.375323,228.154867,216.815


In [3]:
cities = df.columns.tolist()
def myplot(city):
    return df.hvplot(x='DATE', y=city, rot=90, width = (1000), height = (500), title = 'Housing price data by city over the last 10 years')

In [4]:
pn.interact(myplot, city=cities)

Column
    [0] Column
        [0] Select(name='city', options=['Phoenix', 'Atlanta', ...], value='Phoenix')
    [1] Row
        [0] HoloViews(Curve, name='interactive01446')

In [5]:
#In Order to make an Apples to Apples comparison we must convert the data from FRED to a percent change, the dates not alligning correctly will be dealt with below
usa_housing_market = pd.DataFrame(df, columns=['U.S.A'])
usa_house_pct = usa_housing_market.pct_change()
usa_house_pct.reset_index(inplace = True)
usa_house_pct.head()

,DATE,U.S.A
0,2010-01-01,NaN
1,2010-02-01,-0.011968
2,2010-03-01,0.001559
3,2010-04-01,0.003716
4,2010-05-01,-0.000068


In [6]:
from dotenv import load_dotenv
load_dotenv()
#KEYS
alpaca_api_key = ('PKETZDKWTVEJJ92OHW4G')
alpaca_secret_key = ('pFsAVawMY6IEB82q2CwJKhQZi8dDTDywkNjCb5cW')
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [7]:
ticker = ["SPY"]
timeframe = "1D"
start_date = pd.Timestamp('2010-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-05-01', tz='America/New_York').isoformat()

#calling 2010's top ten companies by size ticker
df_spy = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

# Only comparing closing value
df_spy = df_spy.drop(
    columns=['open', 'high', 'low', 'volume'],
    level=1
)
#Allows the data to reflect the average of a month and post it to a day at the at the end for comparison against housing data.
df_spy.index = df_spy.index.date
df_spy.reset_index(inplace = True)
df_spy.rename(columns={'index': 'Date'}, inplace= True)
dfspy = df_spy.set_index('Date')
dfspy.index = pd.to_datetime(dfspy.index)
monthly_spy = dfspy['SPY'].resample('M').mean()
monthly_spy

,close
Date,
2010-01-31,112.409474
2010-02-28,109.161053
2010-03-31,115.464130
2010-04-30,119.845238
2010-05-31,112.794280
...,...
2020-01-31,326.977143
2020-02-29,327.374737
2020-03-31,265.007727


In [8]:
monthly_spy.hvplot(width = (1000), height = (500), title = 'SPY by month for the last decade')

:Curve   [Date]   (close)

In [9]:
#applying the pct_change allows for that apples to apples comparison mentioned above.
spy_pct = monthly_spy.pct_change()
spy_pct.head()

,close
Date,
2010-01-31,NaN
2010-02-28,-0.028898
2010-03-31,0.057741
2010-04-30,0.037943
2010-05-31,-0.058834


In [10]:
#Joining the two dataframes to compare a high level look at the stock market to a high level look at the housing market, the dates need to match up. We will stick with the housing dates for clairty, stock data being 1 day off will not affect accuracy.
Housing_vs_Stocks = pd.concat([usa_house_pct.reset_index(drop=True),spy_pct.reset_index(drop=True)], axis=1)
Housing_vs_Stocks = Housing_vs_Stocks.set_index('DATE')
Housing_vs_Stocks.rename(columns={'U.S.A.':'US Housing Maket','close': 'Stock Index'}, inplace= True)
Housing_vs_Stocks.head()

,U.S.A,Stock Index
DATE,,
2010-01-01,NaN,NaN
2010-02-01,-0.011968,-0.028898
2010-03-01,0.001559,0.057741
2010-04-01,0.003716,0.037943
2010-05-01,-0.000068,-0.058834


In [11]:
#From this graph we can see the stock market has outpaced the housing market on average, despite the volatility.
cumulative_returns = (1 + Housing_vs_Stocks).cumprod()
cumulative_returns.hvplot(y=['U.S.A', 'Stock Index'],width = (1000), height = (500), xticks = (10),
             value_label='Observed Cumulative Returns')

:NdOverlay   [Variable]
   :Curve   [DATE]   (Observed Cumulative Returns)

In [12]:
#From the above graph vlatility appears to be the name of the game for the stock market. Lets double check. 
monthly_std = Housing_vs_Stocks.std()
monthly_std_visule = (1 + monthly_std).cumprod()
monthly_std_visule.hvplot.scatter(title= 'Volatility comparison')

:Scatter   [index]   (0)

In [1]:
def housing_vs_stock():
    """Who performs better"""
    h_v_s = cumulative_returns.hvplot(
        x="DATE",
        y=['U.S.A', 'Stock Index'],
        title = 'Observed Cumulative Returns',
        width = (1000), 
        height = (500),
    ).opts(yformatter="%.0f")
    return h_v_s

Housing_Market = pn.interact(myplot, city=cities)
Monthly_Market_Analysis = pn.Row(housing_vs_stock())

title = pn.pane.Markdown("Housing VS Market")
welcome = pn.pane.Markdown("A simple dashboard dawning complext questions")

Housing_Market = pn.Tabs(
    ('Housing Sales Prive AVG by Month', Housing_Market),
    ('Housing Price Returns VS The Market', Monthly_Market_Analysis),
(
Dashboard=pn.Column(pn.Row(title), Housing_Market, width=900)
Dashboard.servable()

SyntaxError: invalid syntax (<ipython-input-1-b4efb53b7bb9>, line 22)